In [4]:
import os

In [5]:

os.chdir('../')

In [6]:
%pwd


'/Users/ram/Desktop/datascience pro'

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from src.DataScience.constants import *
from src.DataScience.utiles.common import *

In [12]:
class ConfigurationManager:
    def __init__(self, 
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_roots])

    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionconfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config
       

In [16]:
import os
import requests
from urllib.request import urlretrieve
from src.DataScience import logger
import zipfile

In [17]:
class Dataingestion:
    def __init__(self, config: DataIngestionconfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} is downloaded, with following info: {headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists")
    
    def extract_zip_file(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
            logger.info(f"Extracted all the files in {unzip_dir}")

    

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    dataingestion = Dataingestion(config=data_ingestion_config)
    dataingestion.download_file()
    dataingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-21 12:54:39,671: INFO: common:  ymal file:config/config.yaml loaded successfully]
[2025-04-21 12:54:39,676: INFO: common:  ymal file:params.yaml loaded successfully]
[2025-04-21 12:54:39,677: INFO: common:  ymal file:schema.yaml loaded successfully]
[2025-04-21 12:54:39,678: INFO: common:  created directory at: artifacts]
[2025-04-21 12:54:39,679: INFO: common:  created directory at: artifacts/data_ingestion]
[2025-04-21 12:54:40,394: INFO: 3979074966:  artifacts/data_ingestion/data.zip is downloaded, with following info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 14AF:21AE1E:DE26DD:11C75E0:6806784F
Accept-Ranges: bytes
Date: